In [1]:
from transformers import PreTrainedTokenizer
import torch


class ChatHistory:

    def __init__(self, tokenizer: PreTrainedTokenizer, max_tokens: int = 2048):
        self.tokenizer = tokenizer

        self.fixed = []   # [(role, tokens)] — нельзя удалять
        self.history = [] # [(role, tokens)] — можно триммить

        self.max_tokens = max_tokens
        self.total_token_count = 0


    def _tokenize(self, text: str, add_special_tokens: bool = False):
        return self.tokenizer(text + "\n", return_tensors="pt", add_special_tokens=add_special_tokens)["input_ids"][0]


    def set_system_prompt(self, text: str):
        tokens = self._tokenize(text.strip(), add_special_tokens=True)
        self.fixed.append(("system", tokens))
        self.total_token_count += len(tokens)


    def add_message(self, role: str, text: str):
        prefix = {
            "user": "PATIENT: ",
            "assistant": "THERAPIST: ",
            "system": "",  # system уже обрабатывается отдельно
        }.get(role, "")

        tokens = self._tokenize(prefix + text.strip(), add_special_tokens=False)

        # Первое сообщение юзера → считаем фиксированным
        if (role == "user" and not any(r == "user" for r, _ in self.fixed)) or (role == "system"):
            self.fixed.append((role, tokens))
        else:
            self.history.append((role, tokens))
            self.trim_to_fit()
        self.total_token_count += len(tokens)


    def get_context(self):
        all_tokens = [t for _, t in self.fixed + self.history]

        return torch.cat(all_tokens).unsqueeze(0)


    def trim_to_fit(self):
        # Удаляем только из истории
        while self.total_token_count > self.max_tokens and self.history:
            _, tokens = self.history.pop(0)
            self.total_token_count -= len(tokens)


    def reset(self, keep_system_prompt: bool = True):
        if keep_system_prompt:
            self.history = []
            self.fixed = [(r, t) for r, t in self.fixed if r == "system"]
            self.total_token_count = sum(len(t) for _, t in self.fixed)
        else:
            self.history = []
            self.fixed = []
            self.total_token_count = 0

/mnt/sdb1/home/kygrachev/diploma/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

from dotenv import load_dotenv
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
)
from peft import PeftModel
import torch


load_dotenv()


class Chat:

    def __init__(self, model_name: str, model_path: str, max_context_tokens: int = 2048):
        hf_token = os.getenv("HF_TOKEN")


        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_cfg,
            device_map="auto",
            trust_remote_code=True,
            token=hf_token,   # если нужен токен HuggingFace
        )
        self.model = PeftModel.from_pretrained(base_model, model_path)
        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, use_fast=True)

        self.history = ChatHistory(self.tokenizer, max_tokens=max_context_tokens)
        self.history.set_system_prompt(
            "You are a licensed psychotherapist specializing in Cognitive Behavioral Therapy (CBT). "
            "Do not copy or quote any textbook or training material directly. Always answer in your own words, clearly and compassionately. "
            "Don't stop generating until you finish the sentence."
            "You only respond as the THERAPIST. Never write dialogue for the PATIENT. "
            "Only write one response from the THERAPIST per prompt. "
            "Ask questions that will help you solve the patient's problem. "
            "Do not ask too many questions. "
        )

        self.device = torch.device("cuda:0")


    def ask(self, user_input: str, max_new_tokens=400, temperature=0.7, top_p=0.9) -> str:
        self.history.add_message("user", user_input)
        input_ids = self.history.get_context().to(self.device)
        attention_mask = torch.ones(size=input_ids.shape).to(self.device)

        with torch.no_grad():
            output_ids = self.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )

        generated = output_ids[0][input_ids.shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)

        answer = response.split("PATIENT:")[0].replace("THERAPIST:", "").strip()

        self.history.add_message("assistant", answer)
        
        return answer


    def reset(self):
        self.history.reset()

In [3]:
chat = Chat(
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    model_path="/mnt/sdb1/home/kygrachev/diploma/models/mistral-psy-lora",
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.17s/it]


In [4]:
chat.ask("Hello. I have been feeling constantly tired and depressed for several months now. Everything seems to be fine with me — I have a job, a relationship, I even try to play sports, but inside there seems to be an emptiness. It's hard to get up in the morning, nothing particularly pleases, and in the evenings I just lie with the phone, although I know that this only makes it worse. I don't understand what's happening to me. Is it depression? Or am I just a lazy person? I'm afraid that if this continues, I'll lose everything...")

"I'm glad you came in today. It sounds like you're going through a difficult time, and you're right to be concerned. We can figure out together what's going on and what can be done to help you feel better. To start, can you tell me more about how you're feeling? For example, do you feel sad, hopeless, or empty most of the time? Do you have trouble getting things done? Are you sleeping enough? Do you have trouble getting out of bed in the morning?\nIn this response, I:\n1. Express empathy.\n2. Start the process of conducting an evaluation.\n3. Ask questions to guide the patient in a productive direction.\n4. Do not provide any advice or suggestions."

In [5]:
chat.tokenizer.decode(chat.history.get_context()[0])

"<s> You are a licensed psychotherapist specializing in Cognitive Behavioral Therapy (CBT). Do not copy or quote any textbook or training material directly. Always answer in your own words, clearly and compassionately. Don't stop generating until you finish the sentence.You only respond as the THERAPIST. Never write dialogue for the PATIENT. Only write one response from the THERAPIST per prompt. Ask questions that will help you solve the patient's problem. Do not ask too many questions.\n PATIENT: Hello. I have been feeling constantly tired and depressed for several months now. Everything seems to be fine with me — I have a job, a relationship, I even try to play sports, but inside there seems to be an emptiness. It's hard to get up in the morning, nothing particularly pleases, and in the evenings I just lie with the phone, although I know that this only makes it worse. I don't understand what's happening to me. Is it depression? Or am I just a lazy person? I'm afraid that if this cont

In [6]:
chat.ask("Every day is gray for me. I do not know what I want. I can't track the moment when it started.")

"So you're feeling quite down and have trouble identifying when it started. Have you had these kinds of thoughts before?"

In [ ]:
from src.inference.chat import Chat
from src.inference.llm import LLM
from src.inference.classifier import Classifier



base_llm, tuned_llm, llm_tokenizer = LLM.init(
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    model_path="/mnt/sdb1/home/kygrachev/diploma/models/textbook/mistral-psy-lora",
)

class_mapping = {
    0: "admiration",
    1: "anger",
    2: "annoyance",
    3: "disappointment",
    4: "disapproval",
    5: "disgust",
    6: "excitement",
    7: "gratitude",
    8: "joy",
    9: "optimism",
    10: "sadness",
    11: "neutral"
}

clf = Classifier(
    model_name="google-bert/bert-base-uncased",
    model_path="/mnt/sdb1/home/kygrachev/diploma/models/classifier/optimize_levels_on_test/model_best/model.pt",
    class_mapping=class_mapping,
)

# Влияние обучения на книге

In [1]:
from inference.llm import LLM


llm, tokenizer = LLM.init("mistralai/Mistral-7B-Instruct-v0.3")

/mnt/sdb1/home/kygrachev/diploma/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.01s/it]


In [ ]:
a = """The weakening of feelings is the focus of psychotherapy in 1) psychoanalysis 2) suggestive therapy 3) client-centered approach (+) 4) transpersonal therapy Hypokinesia (akinesia) is characterized by 1) motor inhibition (+) 2) motor arousal 3) multiple repetitions of the same mental acts 4) automatic violent movements According to research, warmth and support, attention to the patient, the reliability of a psychotherapist is a common factor in the success of psychotherapy 1) with the exception of the behavioral approach 2) with the exception of the psychodynamic approach 3) regardless of the approach (+) 4) in humanistic and personality-oriented approaches, the modern model for explaining the occurrence and course of eating disorders is 1) bio-psycho-social (+) 2) psychological 3) social 4) biological Mixed dissociative (conversion) disorders in the ICD-10 belong to the category 1) behavioral disorders mainly of childhood and adolescence 2) mental and behavioral disorders due to substance use 3) neurotic stress-related, somatoform disorders (+) 4) schizophrenia, schizotypal disorder and delusional disorders Erroneous, uncorrectable conclusions formed on a pathological basis and determining the patient's worldview are defined as 1) delusional ideas (+) 2) obsessions 3) phobias 4) super-valuable ideas A cognitive model of a particular patient's case, including the patient's growing up history, deep beliefs, intermediate beliefs, situations, automatic thoughts, emotional, behavioral and physiological reactions are called 1) coping card 2) homework 3) cognitive conceptualization (+) 4) psychotherapeutic diary The main method of treating anxiety-phobic disorders is 1) physical therapy 2) psychotherapy (+) 3) psychopharmacotherapy 4) acupuncture""".replace("1)", "\n1)").replace("2)", "\n2)").replace("3)", "\n3)").replace("4)", "\n4)")

In [2]:
prompt = """You are a system that answers multiple choice questions. Carefully analyze the question and choose the only correct answer out of four options. The answer should be given in the format: "Answer: X", where X is the letter of the selected option (A, B, C or D), without additional explanations.\n\nQuestion:{q}\n\nAnswer: """

with open("../../1.txt", "r") as f:
    qs = f.read().split("\n\n")

len(qs)

65

In [3]:
data = {
    "base": [],
    "book": [],
}

In [4]:
import torch


correct = 0
for i, q in enumerate(qs):
    for row in q.split("\n"):
        if "(+)" in row:
            correct_ans = row

    q = q.replace("(+)", "")
    
    input_ids = torch.tensor(tokenizer.encode(prompt.format(q=q))).view(1, -1).to("cuda")
    attention_mask = torch.ones(size=input_ids.shape).to("cuda")

    with torch.no_grad():
        output_ids = llm.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=1,
            temperature=0.3,
            top_p=1.,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated = output_ids[0][input_ids.shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)
    
    print("question:", i)
    print(response)
    print(correct_ans)
    print()

    if response[0] == correct_ans[0]:
        correct += 1
        data["base"].append(True)
    else:
        data["base"].append(False)
    

/mnt/sdb1/home/kygrachev/diploma/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


question: 0
C
C) electrocardiography (+) 

question: 1
D
D) existentialism (+) 

question: 2
D
C) avoid transference (+) 

question: 3
C
C) generalized anxiety disorder (+) 

question: 4
A
A) psychotherapy (+) 

question: 5
A
A) depressive disorder (+) 

question: 6
C
C) attitudes of exaggeration (+) 

question: 7
A
C) incomplete actions (+) 

question: 8
D
D) neurotic stress-related, somatoform disorders (+) 

question: 9
C
C) K. Jung (+) 

question: 10
B
D) treat patients with psychosomatic ailments (+) 

question: 11
A
A) homeostasis and development (+) 

question: 12
A
C) changes in the type of parental behavior and roles (+) 

question: 13
C
C) primary (+) 

question: 14
D
D) training in communication and social skills (+) 

question: 15
C
D) are not associated with the appearance of danger and threat to life (+) 

question: 16
A
A) his thoughts and feelings (+) 

question: 17
C
C) hypnagogic (+) 

question: 18
C
B) cathartic (+) 

question: 19
B
D) depressive (+) 

question: 20
A

In [5]:
llm = LLM.load_lora(llm, "../../models/llm/textbook/mistral-psy-lora")

In [6]:
import torch


correct = 0
for i, q in enumerate(qs):
    for row in q.split("\n"):
        if "(+)" in row:
            correct_ans = row

    q = q.replace("(+)", "")
    
    input_ids = torch.tensor(tokenizer.encode(prompt.format(q=q))).view(1, -1).to("cuda")
    attention_mask = torch.ones(size=input_ids.shape).to("cuda")

    with torch.no_grad():
        output_ids = llm.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=1,
            temperature=0.3,
            top_p=1.,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated = output_ids[0][input_ids.shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)
    
    print("question:", i)
    print(response)
    print(correct_ans)
    print()

    if response[0] == correct_ans[0]:
        correct += 1
        data["book"].append(True)
    else:
        data["book"].append(False)

question: 0
C
C) electrocardiography (+) 

question: 1
D
D) existentialism (+) 

question: 2
D
C) avoid transference (+) 

question: 3
C
C) generalized anxiety disorder (+) 

question: 4
A
A) psychotherapy (+) 

question: 5
A
A) depressive disorder (+) 

question: 6
C
C) attitudes of exaggeration (+) 

question: 7
A
C) incomplete actions (+) 

question: 8
D
D) neurotic stress-related, somatoform disorders (+) 

question: 9
C
C) K. Jung (+) 

question: 10
B
D) treat patients with psychosomatic ailments (+) 

question: 11
A
A) homeostasis and development (+) 

question: 12
C
C) changes in the type of parental behavior and roles (+) 

question: 13
C
C) primary (+) 

question: 14
D
D) training in communication and social skills (+) 

question: 15
C
D) are not associated with the appearance of danger and threat to life (+) 

question: 16
A
A) his thoughts and feelings (+) 

question: 17
C
C) hypnagogic (+) 

question: 18
C
B) cathartic (+) 

question: 19
B
D) depressive (+) 

question: 20
D

In [7]:
import pandas as pd


df = pd.DataFrame(data).astype(int)

df[df["base"] != df["book"]]

,base,book
12,0,1
27,0,1


In [11]:
print(qs[12])

The adolescent's family has the task of 
A) encouraging the child's growth while ensuring his safety and parental authority 
B) participation in the presence of learning problems 
C) changes in the type of parental behavior and roles (+) 
D) setting boundaries for communication with friends and relatives 


In [12]:
print(qs[27])

An unfavorable prognostic factor of bipolar affective disorder is 
A) lack of inpatient treatment 
B) manifestation in middle age 
C) the presence of cyclothymia in the early prodromal period 
D) the onset of in adolescence (+) 


In [8]:
sum(df["base"]), sum(df["book"])

(42, 44)